In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, RidgeCV, LassoCV, LogisticRegression
from sklearn.model_selection import cross_val_score,train_test_split, KFold, cross_val_predict, RepeatedKFold
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc,precision_recall_curve, accuracy_score, \
recall_score, precision_score, confusion_matrix
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier, \
RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
import itertools as it
import time as time
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance

import warnings
from IPython import display
import re

In [2]:
train = pd.read_csv('train_regression.csv')
test = pd.read_csv('test_regression.csv')
train.head()

id    host_id  host_since host_location host_response_time  \
0            53872917  366516391  2018-04-09   Chicago, IL     within an hour   
1            39269663   50276775  2014-12-23           NaN     within an hour   
2  556480733030442463  681680651  2020-03-08           NaN     within an hour   
3            52397280   94345291  2015-10-22   Chicago, IL     within an hour   
4  745949202271858960  790998106  2021-04-04   Chicago, IL     within an hour   

  host_response_rate host_acceptance_rate host_is_superhost  \
0                96%                  95%                 f   
1               100%                  97%                 f   
2               100%                 100%                 f   
3               100%                  98%                 f   
4               100%                 100%                 t   

  host_neighbourhood  host_listings_count  ...  review_scores_checkin  \
0  Back of the Yards                   11  ...                   5.00   
1   Clearwater Beach                 1141  ...                    NaN   
2     Lake View East                    1  ...                   5.00   
3           Bucktown                   55  ...                   4.38   
4    Near South Side                   74  ...                   4.83   

  review_scores_communication review_scores_location review_scores_value  \
0                        5.00                   4.28                4.83   
1                         NaN                    NaN                 NaN   
2                        5.00                   4.93                4.80   
3                        4.23                   4.54                3.85   
4                        4.85                   4.74                4.72   

  instant_bookable  calculated_host_listings_count  \
0                f                               9   
1                t                              58   
2                f                               1   
3                t                              55   
4                t                              74   

   calculated_host_listings_count_entire_homes  \
0                                            1   
1                                            0   
2                                            1   
3                                           55   
4                                           74   

  calculated_host_listings_count_private_rooms  \
0                                            8   
1                                           58   
2                                            0   
3                                            0   
4                                            0   

  calculated_host_listings_count_shared_rooms  reviews_per_month  
0                                           0               0.33  
1                                           0                NaN  
2                                           0               2.78  
3                                           0               3.61  
4                                           0               3.24  

[5 rows x 54 columns]

In [3]:
pd.set_option('display.max_columns', None)

train['price'] = train['price'].str.replace('$', '').str.replace(',', '').astype(float)

train['host_response_rate'] = train['host_response_rate'].str.rstrip('%').astype(float)
train['host_acceptance_rate'] = train['host_acceptance_rate'].str.rstrip('%').astype(float)
train['host_since'] = pd.to_datetime(train['host_since'])

train['host_is_superhost'] = train['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)

train['instant_bookable'] = train['instant_bookable'].apply(lambda x: 1 if x == 't' else 0)
train['host_has_profile_pic'] = train['host_has_profile_pic'].apply(lambda x: 1 if x == 't' else 0)
train['host_identity_verified'] = train['host_identity_verified'].apply(lambda x: 1 if x == 't' else 0)
train['has_availability'] = train['has_availability'].apply(lambda x: 1 if x == 't' else 0)


'------------------------------------------------------------------------------------------------'

test['host_response_rate'] = test['host_response_rate'].str.rstrip('%').astype(float)
test['host_acceptance_rate'] = test['host_acceptance_rate'].str.rstrip('%').astype(float)
test['host_since'] = pd.to_datetime(test['host_since'])

test['instant_bookable'] = test['instant_bookable'].apply(lambda x: 1 if x == 't' else 0)
test['host_has_profile_pic'] = test['host_has_profile_pic'].apply(lambda x: 1 if x == 't' else 0)
test['host_identity_verified'] = test['host_identity_verified'].apply(lambda x: 1 if x == 't' else 0)
test['has_availability'] = test['has_availability'].apply(lambda x: 1 if x == 't' else 0)


# test['host_is_superhost'] = test['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)


# covert host_since into a usuable form
current_date = pd.to_datetime("today")

train['days_since_host_started'] = (current_date - train['host_since']).dt.days
test['days_since_host_started'] = (current_date - test['host_since']).dt.days

# Impute missing values

#Impute numerical columns with the mean
for column in train.select_dtypes(include=['number']).columns:
    train[column].fillna(train[column].mean(), inplace=True)
    
for column in test.select_dtypes(include=['number']).columns:
    test[column].fillna(train[column].mean(), inplace=True)  # Use train's mean to avoid data leakage

# Impute categorical columns with the mode (most frequent value)
for column in train.select_dtypes(include=['object', 'category']).columns:
    train[column].fillna(train[column].mode()[0], inplace=True)
    
for column in test.select_dtypes(include=['object', 'category']).columns:
    test[column].fillna(train[column].mode()[0], inplace=True)  # Use train's mode to avoid data leakage

# print(test.isnull().any())
# print('---------------------------------------------------------')
# print(train.isnull().any())

def extract_bathrooms(text):
    if 'half' in text.lower():
        return 0.5
    else:
        match = re.search(r'\d+\.?\d*', text)
        return float(match.group()) if match else np.nan

train['bathrooms'] = train['bathrooms_text'].apply(extract_bathrooms)
test['bathrooms'] = test['bathrooms_text'].apply(extract_bathrooms)

train['bathrooms'].fillna(train['bathrooms'].mean(), inplace=True)
test['bathrooms'].fillna(test['bathrooms'].mean(), inplace=True)

train.drop(columns=['bathrooms_text', 'host_id', 'host_since', 'first_review', 'last_review'], inplace=True)
test.drop(columns=['bathrooms_text', 'host_id', 'host_since', 'first_review', 'last_review'], inplace=True)

train = pd.get_dummies(train)
test = pd.get_dummies(test)

train.head()

#All columns in the dataset, except id, host_id, host_since, first_review, and last_review have
#to be tried as predictors. (price is the response, so it cannot be used as a predictor either.)

C:\Users\benja\AppData\Local\Temp\ipykernel_26828\2113949972.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  train['price'] = train['price'].str.replace('$', '').str.replace(',', '').astype(float)


id  host_response_rate  host_acceptance_rate  \
0            53872917                96.0                  95.0   
1            39269663               100.0                  97.0   
2  556480733030442463               100.0                 100.0   
3            52397280               100.0                  98.0   
4  745949202271858960               100.0                 100.0   

   host_is_superhost  host_listings_count  host_total_listings_count  \
0                  0                   11                         13   
1                  0                 1141                       2346   
2                  0                    1                          1   
3                  0                   55                         58   
4                  1                   74                         75   

   host_has_profile_pic  host_identity_verified   latitude  longitude  \
0                     1                       1  41.794240 -87.661380   
1                     1                       1  41.892749 -87.620711   
2                     1                       1  41.948160 -87.652309   
3                     1                       1  41.801030 -87.597160   
4                     1                       1  41.855770 -87.624130   

   accommodates  beds   price  minimum_nights  maximum_nights  \
0             1   1.0    30.0              32            1125   
1            12   3.0  1247.0              32             365   
2             6   3.0   214.0               2              45   
3             2   1.0    59.0               2             180   
4             6   3.0   151.0               2             365   

   minimum_minimum_nights  maximum_minimum_nights  minimum_maximum_nights  \
0                      32                      32                    1125   
1                      32                      32                     365   
2                       2                       2                      45   
3                       2                       2                     180   
4                       2                       2                    1125   

   maximum_maximum_nights  minimum_nights_avg_ntm  maximum_nights_avg_ntm  \
0                    1125                    32.0                  1125.0   
1                     365                    32.0                   365.0   
2                      45                     2.0                    45.0   
3                     180                     2.0                   180.0   
4                    1125                     2.0                  1125.0   

   has_availability  availability_30  availability_60  availability_90  \
0                 1               30               60               90   
1                 1               25               49               66   
2                 1                0                0                0   
3                 1               23               53               83   
4                 1                0                0                0   

   availability_365  number_of_reviews  number_of_reviews_ltm  \
0               365                 18                      0   
1               135                  0                      0   
2                 0                 14                     14   
3               355                 13                     13   
4                 0                 64                     42   

   number_of_reviews_l30d  review_scores_rating  review_scores_accuracy  \
0                       0              4.940000                 5.00000   
1                       0              4.759538                 4.79342   
2                       1              4.870000                 5.00000   
3                       3              4.080000                 4.08000   
4                       3              4.800000                 4.74000   

   review_scores_cleanliness  review_scores_checkin  \
0                   4.890000               5.000000   
1                   4.747335  

In [4]:
train_cols = list(train.columns)
test_cols = list(test.columns)

train_cols = [col for col in train_cols if col in test_cols]
test_cols = [col for col in test_cols if col in train_cols]

train_cols == test_cols

True

In [6]:
x_train = train[train_cols].drop(columns=['id'])
y_train = train.price

x_test = test[test_cols].drop(columns=['id'])

In [6]:
base_model = DecisionTreeRegressor(random_state=1)

model = BaggingRegressor(estimator=base_model, n_estimators=10, random_state=1, n_jobs=-1)

model.fit(x_train, y_train)

y_pred = model.predict(x_test)

In [8]:
# coarse grid

base_model = DecisionTreeRegressor(random_state=1) 

model = BaggingRegressor(estimator=base_model, n_estimators=250, random_state=1) 

grid = {

    'max_samples':[0.5, 0.75, 0.9, 1.0], 
    'max_features':[0.5, 0.75, 0.9, 1.0], 
    'bootstrap': [True, False],
    'bootstrap_features': [True, False]
    
}

gscv = GridSearchCV(model, grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train, y_train)

print(gscv.best_params_) # Best hyperparam combos
print(-gscv.best_score_)

y_pred = gscv.predict(x_test)

Fitting 5 folds for each of 64 candidates, totalling 320 fits
{'bootstrap': False, 'bootstrap_features': False, 'max_features': 1.0, 'max_samples': 0.9}
772.4631890243176


In [12]:
# finer grid

base_model = DecisionTreeRegressor(random_state=1) 

model = BaggingRegressor(estimator=base_model, n_estimators=250, random_state=1) 

grid = {

    'max_samples':[0.9, 0.95, 1.0], 
    'max_features':[0.9, 0.95, 1.0], 
    'bootstrap': [False],
    'bootstrap_features': [False]
    
}

gscv = GridSearchCV(model, grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train, y_train)

print(gscv.best_params_) # Best hyperparam combos
print(-gscv.best_score_)

y_pred = gscv.predict(x_test)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
{'bootstrap': False, 'bootstrap_features': False, 'max_features': 1.0, 'max_samples': 0.9}
772.4631890243176


In [16]:
# finest grid

base_model = DecisionTreeRegressor(random_state=1) 

model = BaggingRegressor(estimator=base_model, n_estimators=250, random_state=1) 

grid = {

    'max_samples':[0.9, 0.93, 0.95], 
    'max_features':[0.95, 0.97, 1.0], 
    'bootstrap': [True, False],
    'bootstrap_features': [True, False]
    
}

gscv = GridSearchCV(model, grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train, y_train)

print(gscv.best_params_) # Best hyperparam combos
print(-gscv.best_score_)

y_pred = gscv.predict(x_test)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
{'bootstrap': False, 'bootstrap_features': False, 'max_features': 0.97, 'max_samples': 0.95}
772.4470418926538


In [19]:
# cv settings, shuffle = true

cv_settings = KFold(n_splits=5, shuffle=True, random_state=1)

gscv = GridSearchCV(model, grid, cv=cv_settings, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train, y_train)

print(gscv.best_params_) # Best hyperparam combos
print(-gscv.best_score_)

y_pred = gscv.predict(x_test)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
{'bootstrap': False, 'bootstrap_features': False, 'max_features': 1.0, 'max_samples': 0.95}
774.3601327057447


In [21]:
cv_settings = RepeatedKFold(n_splits=5, n_repeats=5,random_state=1)

gscv = GridSearchCV(model, grid, cv=cv_settings, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train, y_train)

print(gscv.best_params_) # Best hyperparam combos
print(-gscv.best_score_)

y_pred = gscv.predict(x_test)

Fitting 25 folds for each of 36 candidates, totalling 900 fits
{'bootstrap': False, 'bootstrap_features': False, 'max_features': 1.0, 'max_samples': 0.95}
765.1776734611981


In [6]:
# Narrowing features with lasso regression

x_train = train[train_cols].drop(columns=['id'])
y_train = train.price

x_test = test[test_cols].drop(columns=['id'])

sc = StandardScaler()

sc.fit(x_train)
x_train_scaled = sc.transform(x_train)
x_test_scaled = sc.transform(x_test)

lasso_model = Lasso(alpha=0.005, max_iter=400000)

x_train_subset = x_train_scaled[:500, :]

lasso_model.fit(x_train_subset, y_train[:500])

coefficients = pd.DataFrame({
    'feature': x_train.columns,
    'importance': np.abs(lasso_model.coef_)
})

coefficients.sort_values(by='importance', ascending=False, inplace=True)

coefficients


feature  importance
2                   host_listings_count  331.177902
34       calculated_host_listings_count  327.499061
151        host_neighbourhood_Englewood  124.476203
8                          accommodates   90.040897
256         host_neighbourhood_Woodlawn   85.206021
..                                  ...         ...
101          host_location_Wauconda, IL    0.000000
269  neighbourhood_cleansed_Avalon Park    0.000000
99          host_location_Tulum, Mexico    0.000000
272      neighbourhood_cleansed_Beverly    0.000000
190  host_neighbourhood_Marshall Square    0.000000

[380 rows x 2 columns]

In [7]:
important_features = list(coefficients.head(20).feature)

x_train_reduced = x_train[important_features]
x_test_reduced = x_test[important_features]

base_model = DecisionTreeRegressor(random_state=1) 

model = BaggingRegressor(estimator=base_model, n_estimators=250, 
                         random_state=1, bootstrap=False, bootstrap_features=False, 
                         max_features=0.97, max_samples=0.95)

model.fit(x_train_reduced, y_train)

y_pred = model.predict(x_test_reduced)

In [7]:
# descion tree feature selection

x_train = train[train_cols].drop(columns=['id'])
y_train = train.price

x_test = test[test_cols].drop(columns=['id'])

model = DecisionTreeRegressor(random_state=12, max_depth=10, max_leaf_nodes=300)

model.fit(x_train, y_train)

features_importance = pd.DataFrame({'feature': x_train.columns, 'importance': model.feature_importances_})

features_importance.sort_values(by='importance', ascending=False, inplace=True)

features_importance

feature  importance
15                     maximum_maximum_nights    0.932147
257           host_neighbourhood_Wrigleyville    0.048609
40                                  bathrooms    0.006591
22                           availability_365    0.002099
8                                accommodates    0.001993
..                                        ...         ...
147            host_neighbourhood_East Pilsen    0.000000
146         host_neighbourhood_East Hyde Park    0.000000
145          host_neighbourhood_East Downtown    0.000000
144  host_neighbourhood_East Colorado Springs    0.000000
379                     room_type_Shared room    0.000000

[380 rows x 2 columns]

In [8]:
important_features = list(features_importance.head(20).feature)

x_train_reduced = x_train[important_features]
x_test_reduced = x_test[important_features]

base_model = DecisionTreeRegressor(random_state=1) 

model = BaggingRegressor(estimator=base_model, n_estimators=250, 
                         random_state=1, bootstrap=False, bootstrap_features=False, 
                         max_features=0.97, max_samples=0.95)

model.fit(x_train_reduced, y_train)

y_pred = model.predict(x_test_reduced)

In [11]:
# Narrowing features with permutation importance

model = BaggingRegressor(estimator=base_model, n_estimators=250, 
                         random_state=1, bootstrap=False, bootstrap_features=False, 
                         max_features=0.97, max_samples=0.95)

model.fit(x_train_reduced, y_train)

perm_importance = permutation_importance(model, x_train_reduced, y_train, n_repeats=10, random_state=1)

feature_names = [col for col in x_train_reduced.columns if col != 'id' and col != 'price']
features_importance = pd.DataFrame({'feature': feature_names, 'importance': perm_importance.importances_mean})

features_importance.sort_values(by='importance', ascending=False, inplace=True)

features_importance

feature  importance
1                host_neighbourhood_Wrigleyville    6.092900
0                         maximum_maximum_nights    5.789433
2                                      bathrooms    0.017939
3                               availability_365    0.016876
18                                          beds    0.011830
14                             reviews_per_month    0.009117
16                               availability_60    0.008058
4                                   accommodates    0.007634
5                        days_since_host_started    0.004801
8                                      longitude    0.003431
17                     host_total_listings_count    0.002268
7   calculated_host_listings_count_private_rooms    0.002251
15                          review_scores_rating    0.001413
6                     property_type_Entire condo    0.001317
9                     property_type_Entire villa    0.000694
10          host_neighbourhood_Back of the Yards    0.000410
11                        maximum_minimum_nights    0.000394
13                        minimum_nights_avg_ntm    0.000366
19                     room_type_Entire home/apt    0.000234
12               host_neighbourhood_Logan Square    0.000127

In [12]:
important_features = list(features_importance.head(10).feature)

x_train_reduced = x_train_reduced[important_features]
x_test_reduced = x_test_reduced[important_features]

model = BaggingRegressor(estimator=base_model, n_estimators=250, 
                         random_state=1, bootstrap=False, bootstrap_features=False, 
                         max_features=0.97, max_samples=0.95)

model.fit(x_train_reduced, y_train)

y_pred = model.predict(x_test_reduced)

In [13]:
ids = test.id
results = pd.DataFrame({
    'id': ids,
    'predicted': y_pred
})


results.to_csv('baggedTrees_Regression_submission.csv', index=False)
results.head(30)

id  predicted
0   771986218856585018    210.676
1   855276028675941785    126.522
2             48537824    261.132
3             41867473    112.340
4             28361473     93.232
5             51911862     79.064
6   579514121718775234     76.312
7             46242944     33.292
8              5571262     99.676
9   594937541144766707    137.684
10  772410051557583562    206.428
11            13165625     86.144
12            29191606    130.396
13            32381296    428.420
14            50451083    142.920
15            51565135    146.628
16            47176352    122.636
17  924507573112993030     68.476
18            45653951    181.964
19            26795754    107.980
20            42035578     49.712
21            49742536    133.604
22            48932073    118.271
23            15904833    115.180
24  759540854095218350    142.604
25            29885798    146.960
26  655444310450112562    314.084
27            11527162     93.604
28            18735326     97.968
29  606007670793701048    277.852